# Sémantique

_Ismaël Bonneau_

#### Jouons un peu avec la sémantique...

Ce notebook a simplement pour but de jouer avec la sémantique des mots et de prendre en main et comparer plusieurs modèles de connaissance sur la sémantique.

- Le wordnet de NLTK
- Le wordnet de spacy, qui utilise les données du wordnet de NLTK et qui s'installe avec la commande ***pip install spacy-wordnet***. https://spacy.io/
- voir ce qu'on peut imaginer avec des embeddings de mots, qui n'ont rien à voir avec un wordnet mais qui je pense peuvent lui apporter des choses intéressantes.

In [75]:
import os
import ast

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric
from gensim.parsing.preprocessing import strip_multiple_whitespaces, split_alphanum, remove_stopwords

CUSTOM_FILTERS = [strip_tags, strip_punctuation, strip_numeric,
                 strip_multiple_whitespaces, split_alphanum, remove_stopwords]

import spacy
from spacy import displacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

nlp = spacy.load('en_core_web_sm')

dataset_path = "data"

from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("embeddings_wiki2017/model.bin", binary=True)

In [24]:
with open(dataset_path + os.sep + "robust2004.txt", "r") as f:
    queries = ast.literal_eval(f.read())
queries = {d:queries[d][1] for d in queries}

In [51]:
print("query: ", queries["305"])
print("\nquery: ", queries["309"])
print("\nquery: ", queries["401"])

query:  which are the most crashworthy, and least crashworthy,  passenger vehicles?

query:  evidence that rap music has a negative effect on young people

query:  what language and cultural differences impede the integration  of foreign minorities in germany?


## Avec NLTK

In [59]:
from nltk.corpus import wordnet as wn

#### On peut obtenir les Hyponymes: x est un hyponyme de y si x est un type de y.

In [82]:
for word in preprocess_string(queries["305"], CUSTOM_FILTERS):
    if len(wn.synsets(word)) > 0:
        print(word, "hyponyms: ")
        for hyponym in wn.synsets(word)[0].hyponyms():
            print("\t", hyponym)

passenger hyponyms: 
	 Synset('commuter.n.02')
	 Synset('fare.n.03')
	 Synset('hitchhiker.n.01')
	 Synset('stowaway.n.01')
	 Synset('straphanger.n.02')
vehicles hyponyms: 
	 Synset('bumper_car.n.01')
	 Synset('craft.n.02')
	 Synset('military_vehicle.n.01')
	 Synset('rocket.n.01')
	 Synset('skibob.n.01')
	 Synset('sled.n.01')
	 Synset('steamroller.n.02')
	 Synset('wheeled_vehicle.n.01')


#### Pareil pour les Hypernymes: x est un hypernyme de y si y est un type de x (penser à x est la superclasse de y)

In [83]:
for word in preprocess_string(queries["305"], CUSTOM_FILTERS):
    if len(wn.synsets(word)) > 0:
        print(word, "hypernyms: ")
        for hyponym in wn.synsets(word)[0].hypernyms():
            print("\t", hyponym)

passenger hypernyms: 
	 Synset('traveler.n.01')
vehicles hypernyms: 
	 Synset('conveyance.n.03')


### On peut regarder la différence entre les synsets renvoyés par le wordnet et les "mots les plus proches" dans un espace latent d'un word2vec

In [79]:
wn.synsets("vehicle")

[Synset('vehicle.n.01'),
 Synset('vehicle.n.02'),
 Synset('vehicle.n.03'),
 Synset('fomite.n.01')]

In [78]:
model.most_similar("vehicle")

[('truck', 0.7183394432067871),
 ('car', 0.7140822410583496),
 ('Vehicle', 0.6983177661895752),
 ('suv', 0.6757856607437134),
 ('all-terrain', 0.659092903137207),
 ('Vehicles', 0.6529489755630493),
 ('jeep', 0.6424083113670349),
 ('automobile', 0.6405799388885498),
 ('semi-trailer', 0.6346049308776855),
 ('lorry', 0.631385087966919)]

## Avec Spacy

#### On peut générer un arbre syntaxique avec les dépendances

In [43]:
displacy.render(nlp(queries['305']), style="dep", jupyter=True, options={'distance': 90})

In [44]:
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')